In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import numpy as np
from io import BytesIO
from PIL import Image
import requests
import tensorflow as tf
import nest_asyncio
nest_asyncio.apply()

app = FastAPI()
origins = [
    "http://localhost",
    "http://localhost:3000",
]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

model=tf.keras.models.load_model("model3.h5");

CLASS_NAMES = ["Brown_rust", "Healthy", "Yellow_rust"]

@app.get("/")
async def ping():
    return "Alive"


def read_file_as_image(data)->np.ndarray:
    image = np.array(Image.open(BytesIO(data)))
    return image


@app.post("/predict")
async def predict(
        file: UploadFile = File(...)
):
    image = read_file_as_image(await file.read())
    img_batch = np.expand_dims(image, 0)
    predictions=model.predict(img_batch)
    predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
    confidence = np.max(predictions[0])
    print('class',predicted_class,
        'confidence',float(confidence))
    return {
        'class': predicted_class,
        'confidence': float(confidence)
    }

if __name__ == "__main__":
    uvicorn.run(app, host='localhost',port=8000)


INFO:     Started server process [940]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)


1/1 [==============================] - 1s 549ms/step
class Healthy confidence 1.0
INFO:     ::1:63203 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 185ms/step
class Brown_rust confidence 0.9995595812797546
INFO:     ::1:63208 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 69ms/step
class Brown_rust confidence 0.9995595812797546
INFO:     ::1:63502 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 56ms/step
class Brown_rust confidence 0.9995595812797546
INFO:     ::1:63815 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 54ms/step
class Brown_rust confidence 0.9995595812797546
INFO:     ::1:64548 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 62ms/step
class Brown_rust confidence 0.9999868869781494
INFO:     ::1:64681 - "POST /predict HTTP/1.1" 200 OK
INFO:     ::1:64694 - "POST /predict HTTP/1.1" 422 Unprocessable Entity
1/1 [=====================